In [2]:
from tqdm import tqdm_notebook
import json

jloads = json.loads
CHUNK = 102400  # in bytes

def load_data_from_json(subreddits, dappend):
    jloads = json.loads
    for subreddit in subreddits:
        rawfile = open("data/raw_data/{}.json".format(subreddit), "r")
        lines = rawfile.readlines(CHUNK)
        while lines:
            for l in lines:
                try:
                    jline = jloads(l)
                    dappend(jline)
                except Exception as err:
                    logging.error(err)
                    logging.debug(l)
            lines = rawfile.readlines(CHUNK)
        rawfile.close()
        
data = []
load_data_from_json(["nfl", "Overwatch", "kpop"], data.append)
data[:10]

[{'author': 'IMLOwl',
  'author_flair_css_class': 'eagles',
  'author_flair_text': 'Eagles',
  'body': "Over the past few seasons, they've generally been pretty talented at every position but QB, WR, RB and CB. This season, they've added a legitimate receiver and while you can't count on rookies to contribute much, it's hard to imagine being worse than last year at corner. Wentz has potential and if he realizes more of it this year, the offense should be able to compete. I'm not saying it's likely that the Eagles are a SB team or even a contender, but they are a team that can definitely make the playoffs if things break right and once you're in the playoffs, anything can happen over 3 games.",
  'can_gild': True,
  'controversiality': 0,
  'created_utc': 1496275201,
  'distinguished': None,
  'edited': False,
  'gilded': 0,
  'id': 'diandmx',
  'link_id': 't3_6eigk4',
  'parent_id': 't1_diaj75v',
  'retrieved_on': 1498961707,
  'score': 1,
  'stickied': False,
  'subreddit': 'nfl',
  '

# OLD

In [4]:
import copy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from langdetect import detect
import logging

logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

%matplotlib inline

DEBUG:matplotlib:CACHEDIR=/home/jessica/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/jessica/.cache/matplotlib/fontList.json
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [232]:
INPUT_PATH = "data/raw_data/ungilded_sample.csv"
df = pd.read_csv(INPUT_PATH)
df.head(2)

/home/jessica/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5,9,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,id,link_id,parent_id,retrieved_on,score,stickied,subreddit,subreddit_id
0,nanakishi,Owner,C4 Head,/u/azeleon,True,0,1.496275e+09,NaN,False,0,diandlp,t3_6ej12i,t3_6ej12i,1498961706,2,False,RPGStuck_C4,t5_3e5e1
1,lewiscbe,Tesla,NaN,I've got a budget of about $1000 after saving ...,True,0,1.496275e+09,NaN,False,0,diandlq,t3_6eg7ta,t1_diaf78y,1498961706,1,False,ClashRoyale,t5_3bog8


In [247]:
import pickle
with open("cleaned_data.pkl", "wb") as file:
    pickle.dump(df2, file)

In [250]:
#df3 = pd.read_pickle("cleaned_data.pkl")
#df3.head()

df = df2[~df2['author'].isin(BOTS)]
len(df)

276930

## Remove posts made by bots

In [151]:
BOTS = []
with open("data/reddit_bots.txt", "r") as file:
    BOTS = file.readlines()
BOTS = [b.replace("\n", "") for b in BOTS]

In [170]:
num_bot_posts = len(df[df['author'].isin(BOTS)])
print(num_bot_posts, "bot posts")
df = df[~df['author'].isin(BOTS)]

0 bot posts


In [254]:
posts = copy.deepcopy(df['body'])
print(len(posts[posts.isna()]))
posts[posts.isna()]

0


Series([], Name: body, dtype: object)

## Count vectorizer

ungiled_sample[:3000], min_df=2, stop_words='english', algorithm='arpack', NUM_TOPICS=20
* sorts into porn and politics

In [258]:
# Start with a smaller sample
posts = posts[:30000]
vectorizer = CountVectorizer(min_df = 2, stop_words = 'english')
dtm = vectorizer.fit_transform(posts.values)
#cv_df = pd.DataFrame(dtm.toarray(),
#                     index=posts.values,
#                     columns=vectorizer.get_feature_names())
#cv_df.head()

In [259]:
vocab = vectorizer.vocabulary_
sorted_vocab = sorted(vocab.items(), key=(lambda item: item[1]), reverse=True)
sorted_vocab

[('zyra', 22666),
 ('zuckerberg', 22665),
 ('zucker', 22664),
 ('zucchini', 22663),
 ('zsr', 22662),
 ('zoth', 22661),
 ('zoro', 22660),
 ('zooming', 22659),
 ('zoom', 22658),
 ('zoo', 22657),
 ('zoning', 22656),
 ('zones', 22655),
 ('zoned', 22654),
 ('zone', 22653),
 ('zon', 22652),
 ('zombies', 22651),
 ('zombie', 22650),
 ('zombas', 22649),
 ('zomba', 22648),
 ('zodiac', 22647),
 ('zns', 22646),
 ('zlatan', 22645),
 ('zipper', 22644),
 ('zip', 22643),
 ('zionist', 22642),
 ('zinc', 22641),
 ('zimmerman', 22640),
 ('zimmer', 22639),
 ('zimbabwe', 22638),
 ('ziggs', 22637),
 ('ziggler', 22636),
 ('zidane', 22635),
 ('zhuge', 22634),
 ('zhang', 22633),
 ('zfs', 22632),
 ('zeus', 22631),
 ('zetas', 22630),
 ('zeta', 22629),
 ('zest', 22628),
 ('zeroed', 22627),
 ('zero', 22626),
 ('zerker', 22625),
 ('zerk', 22624),
 ('zeppelin', 22623),
 ('zephyr', 22622),
 ('zenyatta', 22621),
 ('zenni', 22620),
 ('zen', 22619),
 ('zelda', 22618),
 ('zeke', 22617),
 ('zed', 22616),
 ('zebra', 22615),

In [265]:
NUM_TOPICS = 20
lsa = TruncatedSVD(NUM_TOPICS, algorithm = 'arpack')
dtm_lsa = lsa.fit_transform(dtm.asfptype())
dtm_lsa

array([[ 9.42693682e-01, -3.95814507e-02, -3.75331878e-02, ...,
        -9.66450082e-01, -2.25787814e-01, -3.69644802e-01],
       [ 9.28899935e-01, -1.36370974e-01, -9.72214243e-02, ...,
        -1.52277358e-01, -7.00678692e-02, -7.84514442e-02],
       [ 4.23790552e-01, -1.46243397e-01,  1.34029213e-01, ...,
         4.91646818e-02,  1.09775891e-01, -1.26450598e-02],
       ...,
       [ 1.70434255e+00, -1.83086270e-01, -6.95190061e-01, ...,
         1.23296018e+00, -1.92074758e-01, -1.20584812e-01],
       [ 2.85519972e-01, -1.37123310e-03, -4.83053152e-03, ...,
        -5.32457872e-01,  5.86989521e-01, -3.30231714e-01],
       [ 2.01645154e+00, -1.96120192e-01,  7.13834243e-02, ...,
        -6.17789935e-01, -1.64003365e-01, -2.73462509e-01]])

In [267]:
topic_probs = pd.DataFrame(dtm_lsa.round(5), index=posts,
                           columns=['t{}'.format(i) for i in range(NUM_TOPICS)])
topic_probs['max'] = topic_probs.idxmax(axis=1)

In [268]:
for i in range(NUM_TOPICS):
    topic = 't{}'.format(i)
    topic_probs[topic_probs['max'] == topic].reset_index()[['body']].to_csv(
        "output/{}.csv".format(topic), index=False)

## TF-IDF